<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [3]</a>'.</span>

In [1]:
# Parameters
RAW_DIR = "data/raw/"
PROCESSED_DIR = "data/processed/"
RANDOM_STATE = 42


In [2]:
# Parameters (papermill)
from pathlib import Path

RAW_DIR = "data/raw"
PROCESSED_DIR = "data/processed"
PROCESSED_LOGS_DIR = "data/processed_logs"
MODELS_DIR = "data/processed/models"
PREDICTIONS_DIR = "data/processed/predictions"
RANDOM_STATE = 42
LOGS_FILE = "logs_normalizados.csv"  

# Normalizar rutas y crear carpetas de salida si no existen
RAW_DIR = Path(RAW_DIR)
PROCESSED_DIR = Path(PROCESSED_DIR)
PROCESSED_LOGS_DIR = Path(PROCESSED_LOGS_DIR)
MODELS_DIR = Path(MODELS_DIR); MODELS_DIR.mkdir(parents=True, exist_ok=True)
PREDICTIONS_DIR = Path(PREDICTIONS_DIR); PREDICTIONS_DIR.mkdir(parents=True, exist_ok=True)

for d in (PROCESSED_DIR, PROCESSED_LOGS_DIR, MODELS_DIR, PREDICTIONS_DIR):
    d.mkdir(parents=True, exist_ok=True)

# Rutas de entrada/salida típicas
ruta_logs = PROCESSED_LOGS_DIR / LOGS_FILE
print("Logs:", ruta_logs.resolve())
print("Predicciones dir:", PREDICTIONS_DIR.resolve())

Logs: /Users/alonso/Downloads/Tesis_SIEM_ML/data/processed_logs/logs_normalizados.csv
Predicciones dir: /Users/alonso/Downloads/Tesis_SIEM_ML/data/processed/predictions


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [3]:
import pandas as pd
import joblib
import os

# RUTAS
ruta_logs = "/Users/alonso/Downloads/Tesis_SIEM_ML/data/processed_logs/logs_normalizados.csv"

# Rutas a modelos y encoders ya entrenados
ruta_modelos = {
    "rf_real": "/Users/alonso/Downloads/Tesis_SIEM_ML/modelos/rf_modelo_supervisado_real.pkl",
    "rf_bal": "/Users/alonso/Downloads/Tesis_SIEM_ML/modelos/rf_modelo_supervisado_balanceado.pkl",
    "if": "/Users/alonso/Downloads/Tesis_SIEM_ML/modelos/modelo_isolation_forest.pkl"
}

ruta_encoders = {
    "rf_real": "/Users/alonso/Downloads/Tesis_SIEM_ML/modelos/label_encoders_supervisado_real.pkl",
    "rf_bal": "/Users/alonso/Downloads/Tesis_SIEM_ML/modelos/label_encoders_supervisado_balanceado.pkl",
    "if": "/Users/alonso/Downloads/Tesis_SIEM_ML/modelos/label_encoders_isolation_forest.pkl"
}

# FUNCIONES AUXILIARES

def aplicar_label_encoding_robusto(df, encoders):
    df_encoded = df.copy()
    for col, encoder in encoders.items():
        df_encoded[col] = df_encoded[col].fillna("").astype(str)
        clases = encoder.classes_
        mapa = {clase: i for i, clase in enumerate(clases)}
        df_encoded[col] = df_encoded[col].apply(lambda val: mapa.get(val, -1))
    return df_encoded

# CARGA DE LOGS
df = pd.read_csv(ruta_logs)

# --- Normaliza nombres de columnas para evitar fallos por mayúsculas/puntos/espacios ---
original_cols = df.columns.tolist()
df.columns = (
    df.columns
      .str.strip()
      .str.lower()
      .str.replace(" ", "_")
)

# Mapea alias conocidos -> nombre canónico
rename_map = {
    "event.channel": "channel",
    "event_channel": "channel",
    "log.channel": "channel",
    "log_level": "log_level",        
    "message": "description",
    "event.original": "description",
    "event_action": "action",
    "event.action": "action",
    "user.name": "src_user",
    "related.user": "src_user",
}
df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns}, inplace=True)

# --- Asegura columnas mínimas ---
required = ["description", "object_type", "channel", "log_level", "action"]
for col in required:
    if col not in df.columns:
        df[col] = "desconocido"  # o pd.NA si prefieres

faltantes = [c for c in required if c not in original_cols and c in df.columns]

# PREDICCIÓN CON TODOS LOS MODELOS 
resultados = {}

for clave in ruta_modelos:
    print(f"=== Procesando modelo: {clave} ===")
    modelo = joblib.load(ruta_modelos[clave])
    encoders = joblib.load(ruta_encoders[clave])

    df_copia = df.copy()
    df_encoded = aplicar_label_encoding_robusto(df_copia, encoders)
    df_encoded = df_encoded[list(encoders.keys())]

    # Hacer predicción
    df[f"pred_{clave}"] = modelo.predict(df_encoded)
    resultados[clave] = df[f"pred_{clave}"].value_counts()

print("Resumen de predicciones:")
for clave, valores in resultados.items():
    print(f"\nModelo: {clave}")
    print(valores)

# (Opcional) Guardar el resultado con las predicciones
df.to_csv(PREDICTIONS_DIR / "resultados_predicciones.csv",
          index=False, encoding="utf-8-sig")
print("\nArchivo guardado como 'resultados_predicciones.csv'")

=== Procesando modelo: rf_real ===


/opt/anaconda3/lib/python3.13/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.7.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/lib/python3.13/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.7.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/opt/anaconda3/lib/python3.13/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 1.7.0 when using version 1.6.1. This might

ValueError: Found array with 0 sample(s) (shape=(0, 6)) while a minimum of 1 is required by RandomForestClassifier.

In [ ]:
# Mostrar ejemplos concretos de predicción
print("\n=== EJEMPLOS DE PREDICCIONES ===")
columnas_pred = ['pred_rf_real', 'pred_rf_bal', 'pred_if']
columnas_mostrar = columnas_pred + [col for col in df.columns if col not in columnas_pred]
df_ejemplos = df[columnas_mostrar].head(5)
print(df_ejemplos.to_string(index=False))

# Mostrar ejemplos positivos detectados por RF real
print("\n=== EJEMPLOS POSITIVOS DETECTADOS POR RF REAL ===")
ej_rf_real = df[df['pred_rf_real'] == 1][columnas_mostrar].head(5)
print(ej_rf_real.to_string(index=False))

# Mostrar ejemplos positivos detectados por RF balanceado
print("\n=== EJEMPLOS POSITIVOS DETECTADOS POR RF BALANCEADO ===")
ej_rf_bal = df[df['pred_rf_bal'] == 1][columnas_mostrar].head(5)
print(ej_rf_bal.to_string(index=False))

# Mostrar ejemplos positivos detectados por Isolation Forest
print("\n=== EJEMPLOS POSITIVOS DETECTADOS POR ISOLATION FOREST ===")
ej_if = df[df['pred_if'] == 1][columnas_mostrar].head(5)
print(ej_if.to_string(index=False))

# Mostrar ejemplos donde los modelos no coinciden
print("\n=== EJEMPLOS DONDE LOS MODELOS NO COINCIDEN ===")
df_diferencias = df[
    (df['pred_rf_real'] != df['pred_rf_bal']) |
    (df['pred_rf_real'] != df['pred_if']) |
    (df['pred_rf_bal'] != df['pred_if'])
]
df_diferencias_mostrar = df_diferencias[columnas_mostrar].head(5)
print(df_diferencias_mostrar.to_string(index=False))

# Guardar ejemplos discrepantes en un archivo CSV para revisión adicional
# df_diferencias_mostrar.to_csv("/Users/alonso/Downloads/Tesis_SIEM_ML/data/ejemplos_discrepantes.csv", index=False)

In [ ]:
print("\n=== INFORMES DETALLADOS CON EXPLICACIÓN DE ANOMALÍA ===")

# Columnas necesarias
columnas_pred = ['pred_rf_real', 'pred_rf_bal', 'pred_if']
columnas_mostrar = columnas_pred + [
    col for col in df.columns if col in ['timestamp', 'src_user', 'action', 'description', 'object_type', 'channel', 'log_level']
]

# Selección de ejemplos
df_ejemplos = df[columnas_mostrar].head(5)

# Revisión de cada evento
for idx, row in df_ejemplos.iterrows():
    print(f"\n EVENTO {idx+1} - ANÁLISIS DETALLADO")
    print(f" Fecha y hora: {row['timestamp']}")
    print(f" Usuario: {row['src_user']}")
    print(f" Acción: {row['action']}")
    print(f" Descripción: {row['description']}")
    print(f" Objeto afectado: {row['object_type']}")
    print(f" Canal: {row['channel']}")
    print(f" Nivel del log: {row['log_level']}")

    print("\n RESPUESTA DE LOS MODELOS:")

    # Random Forest Real
    if row['pred_rf_real'] == 1:
        print(" [RF Real] Detectó posible AMENAZA basada en datos reales.")
    else:
        print(" [RF Real] No considera este evento riesgoso.")

    # Random Forest Balanceado
    if row['pred_rf_bal'] == 1:
        print(" [RF Balanceado] También lo detectó como AMENAZA.")
    else:
        print(" [RF Balanceado] Evento clasificado como normal.")

    # Isolation Forest
    if row['pred_if'] in [1, -1]:
        print(" [Isolation Forest] Detectó una ANOMALÍA.")
        print(" Posible explicación:")

        # Explicación basada en heurística
        if pd.isna(row['src_user']) or '$' in str(row['src_user']):
            print("    • El evento fue ejecutado por una cuenta de sistema o servicio, lo cual puede ocultar actividad maliciosa.")
        if "credential" in str(row['action']).lower():
            print("    • Se accedió al gestor de credenciales, lo que suele ser inusual en logs normales.")
        if "permissions" in str(row['action']).lower():
            print("    • Hubo un cambio de permisos, lo cual puede estar asociado a escalamiento de privilegios.")
        if row['log_level'] in ['critical', 'error']:
            print("    • El nivel del log es alto, indicando un posible incidente o fallo importante.")
        if row['channel'] not in ['System', 'Security']:
            print("    • El evento fue registrado en un canal poco común.")
        if str(row['object_type']).strip() == '-' or pd.isna(row['object_type']):
            print("    • No se especifica el tipo de objeto afectado, lo cual puede indicar evasión de registro.")

    else:
        print(" [Isolation Forest] No detectó ninguna anomalía.")

    print("\n CONCLUSIÓN:")
    if row['pred_rf_real'] == 1 or row['pred_rf_bal'] == 1 or row['pred_if'] in [1, -1]:
        print(" Este evento requiere revisión humana. Múltiples señales apuntan a comportamiento sospechoso.")
    else:
        print(" El evento es considerado normal por todos los modelos.")

print("\n Fin del informe detallado.")